# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp=hgh/lda/si-q4)
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic(1)
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree(1)
          

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.122682e+02     1.652504e+00
 * time: 0.8273241519927979
     1     1.088194e+01     9.293316e-01
 * time: 2.25628399848938
     2    -1.213888e+01     1.086085e+00
 * time: 2.922645092010498
     3    -3.452989e+01     8.379984e-01
 * time: 3.8980581760406494
     4    -4.827546e+01     5.582063e-01
 * time: 4.873944044113159
     5    -5.710443e+01     1.874320e-01
 * time: 5.8821680545806885
     6    -5.978810e+01     9.497801e-02
 * time: 6.527482032775879
     7    -6.076928e+01     5.528690e-02
 * time: 7.171008110046387
     8    -6.119773e+01     7.142007e-02
 * time: 7.842709064483643
     9    -6.155908e+01     3.362932e-02
 * time: 8.568040132522583
    10    -6.182108e+01     2.628880e-02
 * time: 9.212083101272583
    11    -6.199024e+01     2.145215e-02
 * time: 9.874828100204468
    12    -6.206437e+01     1.990505e-02
 * time: 10.520344972610474
    13    -6.212278e+01     1.457103e-02
 * time: 11.18025517463684
    

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671078
    AtomicLocal         -18.8557717
    AtomicNonlocal      14.8522666
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485386 
    Xc                  -19.3336826

    total               -62.261666456197